In [30]:
from vnx_utils import *
import pynq
import numpy as np
import os
import time
from collections import namedtuple
import yaml

PROJECT_ROOT = "../"
XCLBINS_DIR = "/home/aliu/xclbins"

In [31]:
XCLBINS_DIR = os.path.join(XCLBINS_DIR, "univmon-redo")
manifests_file_path = os.path.join(PROJECT_ROOT, "tools/manifests/ground_truth_univmon-redo.yml")
manifests = None
with open(manifests_file_path) as f:
    manifests = yaml.safe_load(f)

if(manifests is None):
    raise Exception("Manifest is None")

xclbin_recv = os.path.join(PROJECT_ROOT, 'benchmark.intf1.xilinx_u280_xdma_201920_3/vnx_benchmark_if1.xclbin')

header = ["sk_name", "levels", "rows", "logcols", "hash_units", "mpps"]
ManifestEntry = namedtuple("ManifestEntry", header)

In [32]:
entry_list = []

def get_throughput(xclbin, sketch_manifest):
    assert(len(pynq.Device.devices) == 2)
    print("Found the following NIC(s)")
    for i in range(len(pynq.Device.devices)):
        print("{}) {}".format(i, pynq.Device.devices[i].name))
    
    workers = pynq.Device.devices
    print("Loading xclbin to sender")
    ol_w0 = pynq.Overlay(xclbin, device=workers[0])
    print("Loading xclbin to reciever")
    ol_w1 = pynq.Overlay(xclbin_recv, device=workers[1])
    
    print("Link worker 0 {}; link worker 1 {}".format(ol_w0.cmac_1.linkStatus(),ol_w1.cmac_1.linkStatus()))
    
    print("Configuring sender")
    print(ol_w1.networklayer_1.updateIPAddress('192.168.0.10', debug=True))
    #2
    ol_w1.networklayer_1.sockets[1] = ('192.168.0.5', 62177, 60512, True)
    ol_w1.networklayer_1.populateSocketTable()
    #3 
    ol_w1.networklayer_1.arpDiscovery()
    #4
    print(ol_w1.networklayer_1.readARPTable())
    
    print("Configuring reciever")
    print(ol_w0.networklayer_1.getNetworkInfo())
    #2
    ol_w0.networklayer_1.sockets[7] = ('192.168.0.10', 60512, 62177, True)
    ol_w0.networklayer_1.populateSocketTable()
    #3 
    ol_w0.networklayer_1.arpDiscovery()
    #4
    ol_w0.networklayer_1.readARPTable()
    
    print("Configuring recieving application")
    ol_w1_tg = ol_w1.traffic_generator_1_1
    ol_w1_tg.register_map.debug_reset = 1
    ol_w1_tg.register_map.mode = benchmark_mode.index('CONSUMER')
    ol_w1_tg.register_map.CTRL.AP_START = 1
    
    num_packets = 1000_000
    
    # Checking if using gmem
    using_gmem = False
    sketch_buf = None
    sketch_wh = None
    if('logcols_emem' in sketch_manifest):
        using_gmem = True
        print("Setting up memory buffers")
        sketch_kernel = ol_w0.update_sketch_1
        r = int(sketch_manifest['rows'])
        lce = int(sketch_manifest['logcols_emem'])
        ce = (1<<lce)
        size = r * ce
        shape = (r, ce)
        sketch_buf = pynq.allocate(shape, dtype=np.uint32, target=ol_w0.HBM1)
        sketch_wh = sketch_kernel.start(sketch_buf, num_packets)
    
    print("Configuring sending application")
    freq = 292
    ol_w0_tg = ol_w0.traffic_generator_1_3
    ol_w0_tg.register_map.mode = benchmark_mode.index('PRODUCER')
    ol_w0_tg.register_map.dest_id = 7
    ol_w1_tg.freq = freq
    ol_w0_tg.freq = freq

    print("Starting to send packets")
    for pkt in [num_packets]:
        ol_w0_tg.register_map.debug_reset = 1
        ol_w1_tg.register_map.debug_reset = 1
        ol_w0_tg.register_map.time_between_packets = 0
        ol_w0_tg.register_map.number_packets = pkt
        for i in range(1):
            beats = i + 1
            ol_w0_tg.register_map.number_beats = beats
            ol_w0_tg.register_map.CTRL.AP_START = 1
            while int(ol_w0_tg.register_map.out_traffic_packets) != pkt:
                print("Packets sent till now: ", int(ol_w0_tg.register_map.out_traffic_packets))
                time.sleep(0.8)
            # Get results from local and remote worker
            rx_tot_pkt, rx_thr, rx_time = ol_w1_tg.computeThroughputApp('rx')
            tx_tot_pkt, tx_thr, tx_time = ol_w0_tg.computeThroughputApp('tx')
            # Create dict entry for this particular experiment
            entry_dict = {'size': (beats * 64), 'rx_pkts' : rx_tot_pkt, 'tx_thr': tx_thr, 'rx_thr': rx_thr}
            entry_dict['xclbin'] = xclbin
            entry_list.append(entry_dict)
            throughput = pkt / (rx_time * 1000_000)
            # Reset probes to prepare for next computation
            ol_w0_tg.resetProbes()
            ol_w1_tg.resetProbes() 
            print("Sent {:14,} size: {:4}-Byte done!\tGot {:14,} took {:8.4f} sec, thr: {:.3f} Gbps"\
                  .format(pkt,beats*64, rx_tot_pkt, rx_time, rx_thr))
            time.sleep(0.5)
    
    if(using_gmem):
        del sketch_buf
    
    pynq.Overlay.free(ol_w0)
    pynq.Overlay.free(ol_w1)
    return throughput

In [33]:
prefix = "benchmark.intf1.sketch1_"
suffix = ".xilinx_u280_xdma_201920_3"
binary_file_name = "vnx_benchmark_if1.xclbin"
sketch2tag = {
    'COUNT_MIN_SKETCH': "cm",
    'COUNT_SKETCH': "cs",
    'UNIVMON': "univmon"
}
csvname = {
    'COUNT_MIN_SKETCH': "cm-sketch",
    'COUNT_SKETCH': "count-sketch",
    'UNIVMON': "univmon"
}
results = []
for manifest in manifests:
    sketch = manifest['sketches'][0]
    sname = 'COUNT_MIN_SKETCH'
    if('sketch_name' in sketch):
        sname = sketch['sketch_name']
    univmon_levels = 16
    if(sname == 'UNIVMON' and 'univmon_levels' in sketch):
        univmon_levels = sketch['univmon_levels']
    r = sketch['rows']
    c = sketch['logcols']
    h = sketch['hash_units']
    tag = "{}_r{}_c{}_h{}".format(sketch2tag[sname], r, c, h)
    if(sname == 'UNIVMON'):
        tag = "{}_l{}_r{}_c{}_h{}".format(sketch2tag[sname], univmon_levels, r, c, h)
    # Deprecated:
    if('logcols_emem' in sketch):
        lce = sketch['logcols_emem']
        tag = "r{}_c{}_e{}_h{}".format(r, c, lce, h)
    binary_file_dir = prefix + tag + suffix
    binary_file_path = os.path.join(
        XCLBINS_DIR, os.path.join(binary_file_dir, binary_file_name))
    print(binary_file_path)
    if(os.path.isfile(binary_file_path)):
        thr = get_throughput(binary_file_path, sketch)
        results.append(ManifestEntry(csvname[sname], univmon_levels, r, c, h, thr))

/home/aliu/xclbins/univmon-redo/benchmark.intf1.sketch1_univmon_l16_r3_c8_h4.xilinx_u280_xdma_201920_3/vnx_benchmark_if1.xclbin
Found the following NIC(s)
0) xilinx_u280_xdma_201920_3
1) xilinx_u280_xdma_201920_3
Loading xclbin to sender
Loading xclbin to reciever
Link worker 0 {'cmac_link': True}; link worker 1 {'cmac_link': True}
Configuring sender
{'HWaddr': '00:0a:35:02:9d:0a', 'inet addr': '192.168.0.10', 'gateway addr': '192.168.0.1', 'Mask': '255.255.255.0'}
Position   5	MAC address 00:0a:35:02:9d:e5	IP address 192.168.0.5
None
Configuring reciever
{'HWaddr': '00:0a:35:02:9d:e5', 'inet addr': '192.168.0.5', 'gateway addr': '192.168.0.1', 'Mask': '255.255.255.0'}
Position  10	MAC address 00:0a:35:02:9d:0a	IP address 192.168.0.10
Configuring recieving application
Configuring sending application
Starting to send packets
Packets sent till now:  5780
Sent      1,000,000 size:   64-Byte done!	Got      1,000,000 took   0.0109 sec, thr: 46.816 Gbps
/home/aliu/xclbins/univmon-redo/benchm

In [34]:
# CSV
print(", ".join(header))
for entry in results:
    vals = [str(getattr(entry, hdr)) for hdr in header]
    print(", ".join(vals))

sk_name, levels, rows, logcols, hash_units, mpps
univmon, 16, 3, 8, 4, 91.43807669399997
univmon, 4, 6, 6, 4, 89.8462091361287
univmon, 8, 9, 8, 4, 61.473787745326725
